<a href="https://colab.research.google.com/github/TatyanaPythonista/clinical-trials/blob/main/parexel_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Definitions**

An **odds ratio (OR)** is a measure of association between a certain property A and a second property B in a population.

**Confidence interval** is a range of values so defined that there is a specified probability that the value of a parameter lies within it.

**P-value** is the probability that a particular statistical measure, such as the mean or standard deviation, of an assumed probability distribution will be greater than or equal to (or less than or equal to in some instances) observed results.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

Importing the required libraries


In [ ]:
from google.colab import files

import pandas as pd
import numpy as np
import scipy

import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# **Function to count statistics**

In [ ]:
def proportions_diff_confint_ind(sample1, sample2, n1, n2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sample1) / n1
    p2 = float(sample2) / n2
    
    fuctor = np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)

    left_boundary = (p1 - p2) - z * fuctor
    right_boundary = (p1 - p2) + z * fuctor
    
    return (left_boundary, right_boundary)

In [ ]:
def proportions_diff_z_stat_ind(sample1, sample2, n1, n2):
    
    p1 = float(sample1) / n1
    p2 = float(sample2) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [ ]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

# Data 

In [ ]:
files.upload()

Saving resp1.csv to resp1.csv


{'resp1.csv': b'gender@SITEID@SUBJID@TRTPN@responseCategory\nMALE@0001@027@2@SD\nFEMALE@0001@039@1@PD\nMALE@0001@126@2@PD\nMALE@0001@154@1@SD\nFEMALE@0001@161@1@PD\nFEMALE@0001@198@1@SD\nMALE@0001@221@1@SD\nFEMALE@0001@280@1@SD\nMALE@0001@540@1@SD\nMALE@0001@557@1@SD\nFEMALE@0001@599@2@SD\nMALE@0001@636@2@SD\nFEMALE@0010@152@1@SD\nFEMALE@0010@267@2@PD\nFEMALE@0010@466@1@PD\nMALE@0010@568@2@NE\nFEMALE@0010@572@1@PD\nMALE@0100@053@2@SD\nMALE@0100@058@1@PD\nFEMALE@0100@114@2@SD\nMALE@0100@457@1@SD\nMALE@0102@074@2@NE\nMALE@0102@090@1@PD\nMALE@0102@230@1@SD\nMALE@0102@255@2@SD\nFEMALE@0102@272@2@PD\nMALE@0102@365@2@SD\nMALE@0102@366@2@PR\nFEMALE@0102@439@1@PD\nFEMALE@0102@471@2@SD\nMALE@0102@474@1@PD\nMALE@0102@496@2@SD\nFEMALE@0102@502@1@PR\nFEMALE@0102@729@2@PR\nMALE@0102@764@2@SD\nMALE@0103@035@2@SD\nMALE@0103@092@2@PR\nFEMALE@0103@265@2@PR\nFEMALE@0103@344@1@PD\nFEMALE@0103@573@1@PD\nMALE@0103@659@2@PD\nFEMALE@0103@725@1@PD\nMALE@0103@745@2@SD\nFEMALE@0104@142@1@PD\nFEMALE@0104@391@2@S

In [ ]:
data = pd.read_csv('resp1.csv', sep='@')

In [ ]:
data.head(10)

,gender,SITEID,SUBJID,TRTPN,responseCategory
0,MALE,1,27,2,SD
1,FEMALE,1,39,1,PD
2,MALE,1,126,2,PD
3,MALE,1,154,1,SD
4,FEMALE,1,161,1,PD
5,FEMALE,1,198,1,SD
6,MALE,1,221,1,SD
7,FEMALE,1,280,1,SD
8,MALE,1,540,1,SD
9,MALE,1,557,1,SD


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582 entries, 0 to 581
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   gender            582 non-null    object
 1   SITEID            582 non-null    int64 
 2   SUBJID            582 non-null    int64 
 3   TRTPN             582 non-null    int64 
 4   responseCategory  582 non-null    object
dtypes: int64(3), object(2)
memory usage: 22.9+ KB


In [ ]:
data.describe()

,SITEID,SUBJID,TRTPN
count,582.00000,582.000000,582.000000
mean,59.42268,402.443299,1.498282
std,42.34690,231.560424,0.500427
min,1.00000,1.000000,1.000000
25%,23.00000,197.250000,1.000000
50%,52.00000,408.000000,1.000000
75%,99.00000,599.750000,2.000000
max,141.00000,797.000000,2.000000


In [ ]:
data.shape

(582, 5)

# Preparation

In [ ]:
data.drop(columns=['SITEID', 'SUBJID'], inplace=True)
data.columns = ['gender',	'treatment',	'was_response']
data['gender'] = (data['gender'] == 'MALE').astype(int)
data.treatment.replace({1: 0, 2: 1}, inplace=True)
data.was_response.replace({'CR': 1, 'PR': 1, 'SD': 0, 'PD': 0, 'NE': 0}, inplace=True)
data['intercept'] = 1

In [ ]:
data.head()

,gender,treatment,was_response,intercept
0,1,1,0,1
1,0,0,0,1
2,1,1,0,1
3,1,0,0,1
4,0,0,0,1


In [ ]:
data.shape

(582, 4)

# Statistical findings

*   H0 : Response is independent on a treatment.
*   H1 : Response is dependent on a treatment.

In [ ]:
no_resp_treat1 = data.loc[data['treatment'] == 0].was_response.value_counts()[0]
resp_treat1 = data.loc[data['treatment'] == 0].was_response.value_counts()[1]
no_resp_treat2 = data.loc[data['treatment'] == 1].was_response.value_counts()[0]
resp_treat2 = data.loc[data['treatment'] == 1].was_response.value_counts()[1]
total_treat1 = no_resp_treat1 + resp_treat1
total_treat2 = no_resp_treat2 + resp_treat2

In [ ]:
print(f'95% confidence interval for a difference between proportions \
({proportions_diff_confint_ind(resp_treat1, resp_treat2, total_treat1, total_treat2, alpha = 0.05)[0]:.4f}:\
{proportions_diff_confint_ind(resp_treat1, resp_treat2, total_treat1, total_treat2, alpha = 0.05)[1]:.4f})')

95% confidence interval for a difference between proportions (0.0087:0.1266)


In [ ]:
print(resp_treat1, resp_treat2, total_treat1, total_treat2)

56 36 292 290


In [ ]:
print(f"p-value: {proportions_diff_z_test(proportions_diff_z_stat_ind(resp_treat1, resp_treat2, total_treat1, total_treat2)):.2f}")

p-value: 0.03


# Result: reject the null hypothesis, p-value < 0.05.

# Creation of logistic regression model with one variable

Create a model of logistic regression by statsmodels library.


In [ ]:
z = scipy.stats.norm.ppf(1 - 0.05 / 2.)

In [ ]:
X_1 = data.drop(columns=['was_response', 'gender'])
y_1 = data.was_response

In [ ]:
logit_model_1 = sm.Logit(y_1, X_1)
result_1 = logit_model_1.fit()
print(result_1.summary())

Optimization terminated successfully.
         Current function value: 0.432137
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:           was_response   No. Observations:                  582
Model:                          Logit   Df Residuals:                      580
Method:                           MLE   Df Model:                            1
Date:                Thu, 30 Dec 2021   Pseudo R-squ.:                0.009915
Time:                        01:40:09   Log-Likelihood:                -251.50
converged:                       True   LL-Null:                       -254.02
Covariance Type:            nonrobust   LLR p-value:                   0.02481
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
treatment     -0.5153      0.232     -2.222      0.026      -0.970      -0.061
intercept     -1.4385      0.

In [ ]:
result_1.params.values

array([-0.51533521, -1.43848011])

In [ ]:
a1 = result_1.params.values[0]
a0 = result_1.params.values[1]

In [ ]:
def func1(sex):
  func1 = a0 + a1*sex
  return func1

In [ ]:
OR_t1vst2 = np.exp(func1(sex=0))/np.exp(func1(sex=1))
left_OR_t1vst2 = np.exp((func1(sex=0))-(func1(sex=1)) - z*0.232)
right_OR_t1vst2 = np.exp((func1(sex=0))-(func1(sex=1)) + z*0.232)

In [ ]:
print(f'An odds ratio of treatment 1 over treatment 2 is {OR_t1vst2:.2f} with CI ({left_OR_t1vst2:.2f}, {right_OR_t1vst2:.2f})')

An odds ratio of treatment 1 over treatment 2 is 1.67 with CI (1.06, 2.64)


Count odds ratio using Table2x2


In [ ]:
table = sm.stats.Table2x2(np.array([[resp_treat1, no_resp_treat1], [resp_treat2, no_resp_treat2]]))

In [ ]:
print(table.summary(method='normal'))

               Estimate   SE   LCB   UCB  p-value
-------------------------------------------------
Odds ratio        1.674       1.063 2.638   0.026
Log odds ratio    0.515 0.232 0.061 0.970   0.026
Risk ratio        1.545       1.050 2.272   0.027
Log risk ratio    0.435 0.197 0.049 0.821   0.027
-------------------------------------------------


In [ ]:
a = table.summary(method='normal')

In [ ]:
print(f'An odds ratio of treatment 1 over treatment 2 is {np.exp(0.515):.2f} \
with CI ({np.exp(0.515 - z*0.232):.2f}, {np.exp(0.515 + z*0.232):.2f})')

An odds ratio of treatment 1 over treatment 2 is 1.67 with CI (1.06, 2.64)


# Creation of logistic regression model with three variable
# Preparation

In [ ]:
data['interaction'] = data['gender']*data['treatment']

In [ ]:
data.head()

,gender,treatment,was_response,intercept,interaction
0,1,1,0,1,1
1,0,0,0,1,0
2,1,1,0,1,1
3,1,0,0,1,0
4,0,0,0,1,0


Create a model of logistic regression by statsmodels library.

In [ ]:
X_2 = data.drop(columns=['was_response'])
y_2 = data.was_response

In [ ]:
logit_model_2 = sm.Logit(y_2, X_2)
result_2 = logit_model_2.fit()
print(result_2.summary())

Optimization terminated successfully.
         Current function value: 0.431351
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:           was_response   No. Observations:                  582
Model:                          Logit   Df Residuals:                      578
Method:                           MLE   Df Model:                            3
Date:                Thu, 30 Dec 2021   Pseudo R-squ.:                 0.01172
Time:                        01:48:03   Log-Likelihood:                -251.05
converged:                       True   LL-Null:                       -254.02
Covariance Type:            nonrobust   LLR p-value:                    0.1139
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
gender          0.2707      0.300      0.903      0.366      -0.317       0.858
treatment      -0.3067    

In [ ]:
result_2.params.values

array([ 0.27073251, -0.30673027, -1.5841201 , -0.38136214])

In [ ]:
b0 = result_2.params.values[2]
b1 = result_2.params.values[0]
b2 = result_2.params.values[1]
b3 = result_2.params.values[3]

In [ ]:
def func2(sex, treatment):
  func = b0 + b1*sex + b2*treatment + b3*sex*treatment
  return func

In [ ]:
OR_t1vst2_m = np.exp(func2(sex=1, treatment=0))/np.exp(func2(sex=1, treatment=1))
left_OR_t1vst2_m = np.exp((func2(sex=1, treatment=0))-(func2(sex=1, treatment=1)) - z*0.349)
right_OR_t1vst2_m = np.exp((func2(sex=1, treatment=0))-(func2(sex=1, treatment=1)) + z*0.349)

In [ ]:
print(f'Odds ratio of treatment 1 over treatment 2 | Male is {OR_t1vst2_m:.2f} with CI ({left_OR_t1vst2_m:.2f}, {right_OR_t1vst2_m:.2f})')

Odds ratio of treatment 1 over treatment 2 | Male is 1.99 with CI (1.00, 3.94)


In [ ]:
OR_t1vst2_f = np.exp(func2(sex=0, treatment=0))/np.exp(func2(sex=0, treatment=1))
left_OR_t1vst2_f = np.exp((func2(sex=0, treatment=0))-(func2(sex=0, treatment=1)) - z*0.349)
right_OR_t1vst2_f = np.exp((func2(sex=0, treatment=0))-(func2(sex=0, treatment=1)) + z*0.349)

In [ ]:
print(f'Odds ratio of treatment 1 over treatment 2 | Female is {OR_t1vst2_f:.2f} with CI ({left_OR_t1vst2_f:.2f}, {right_OR_t1vst2_f:.2f})')

Odds ratio of treatment 1 over treatment 2 | Female is 1.36 with CI (0.69, 2.69)


In [ ]:
OR_FvsM_t1 = np.exp(func2(0, 0))/np.exp(func2(1, 0))
left_OR_FvsM_t1 = np.exp((func2(0, 0))-(func2(1, 0)) - z*0.300)
right_OR_FvsM_t1 = np.exp((func2(0, 0))-(func2(1, 0)) + z*0.300)

In [ ]:
print(f'Odds ratio of Female over Male | treatment 1 is {OR_FvsM_t1:.2f} with CI ({left_OR_FvsM_t1:.2f}, {right_OR_FvsM_t1:.2f})')

Odds ratio of Female over Male | treatment 1 is 0.76 with CI (0.42, 1.37)


In [ ]:
OR_FvsM_t2 = np.exp(func2(0, 1))/np.exp(func2(1, 1))
left_OR_FvsM_t2 = np.exp((func2(0, 1))-(func2(1, 1)) - z*0.300)
right_OR_FvsM_t2 = np.exp((func2(0, 1))-(func2(1, 1)) + z*0.300)

In [ ]:
print(f'Odds ratio of Female over Male | treatment 2 is {OR_FvsM_t2:.2f} with CI ({left_OR_FvsM_t2:.2f}, {right_OR_FvsM_t2:.2f})')

Odds ratio of Female over Male | treatment 2 is 1.12 with CI (0.62, 2.01)


# Result from regresion model


*   Odds ratio of treatment 1 over treatment 2 | Male is 1.99 with CI (1.00, 3.94)

*   Odds ratio of treatment 1 over treatment 2 | Female is 1.36 with CI (0.69, 2.69)

*   Odds ratio of Female over Male | treatment 1 is 0.76 with CI (0.42, 1.37)

*   Odds ratio of Female over Male | treatment 2 is 1.12 with CI (0.62, 2.01)


Count odds ratio using Table2x2

In [ ]:
data_gender1 = data.loc[data['gender'] == 1] # male

In [ ]:
no_resp_m_treat1 = data_gender1.loc[data['treatment'] == 0].was_response.value_counts()[0]
no_resp_m_treat2 = data_gender1.loc[data['treatment'] == 1].was_response.value_counts()[0]
resp_m_treat1 = data_gender1.loc[data['treatment'] == 0].was_response.value_counts()[1]
resp_m_treat2 = data_gender1.loc[data['treatment'] == 1].was_response.value_counts()[1]

In [ ]:
print(no_resp_m_treat1, no_resp_m_treat2, resp_m_treat1, resp_m_treat2)

119 148 32 20


In [ ]:
table2 = sm.stats.Table2x2(np.array([[resp_m_treat1, no_resp_m_treat1], [resp_m_treat2, no_resp_m_treat2]]))

In [ ]:
print(table2.summary(method='normal'))

               Estimate   SE   LCB   UCB  p-value
-------------------------------------------------
Odds ratio        1.990       1.083 3.657   0.027
Log odds ratio    0.688 0.310 0.080 1.297   0.027
Risk ratio        1.780       1.065 2.975   0.028
Log risk ratio    0.577 0.262 0.063 1.090   0.028
-------------------------------------------------


In [ ]:
print(f'Odds ratio of treatment 1 vs treatment 2 | Male is {np.exp(0.688):.2f} with ({np.exp(0.688 - z*0.310):.2f}, {np.exp(0.688 + z*0.310):.2f})')

Odds ratio of treatment 1 vs treatment 2 | Male is 1.99 with (1.08, 3.65)


In [ ]:
data_gender0 = data.loc[data['gender'] == 0] # female

In [ ]:
no_resp_f_treat1 = data_gender0.loc[data['treatment'] == 0].was_response.value_counts()[0]
no_resp_f_treat2 = data_gender0.loc[data['treatment'] == 1].was_response.value_counts()[0]
resp_f_treat1 = data_gender0.loc[data['treatment'] == 0].was_response.value_counts()[1]
resp_f_treat2 = data_gender0.loc[data['treatment'] == 1].was_response.value_counts()[1]

In [ ]:
print(no_resp_f_treat1, no_resp_f_treat2, resp_f_treat1, resp_f_treat2)

117 106 24 16


In [ ]:
table3 = sm.stats.Table2x2(np.array([[resp_f_treat1, no_resp_f_treat1], [resp_f_treat2, no_resp_f_treat2]]))

In [ ]:
print(table3.summary(method='normal'))

               Estimate   SE   LCB    UCB  p-value
--------------------------------------------------
Odds ratio        1.359        0.685 2.696   0.380
Log odds ratio    0.307 0.349 -0.378 0.992   0.380
Risk ratio        1.298        0.724 2.328   0.382
Log risk ratio    0.261 0.298 -0.324 0.845   0.382
--------------------------------------------------


In [ ]:
print(f'Odds ratio of treatment 1 vs treatment 2 | Female {np.exp(0.307):.2f} with ({np.exp(0.307 - z*0.349):.2f}, {np.exp(0.307 + z*0.349):.2f})')

Odds ratio of treatment 1 vs treatment 2 | Female 1.36 with (0.69, 2.69)


In [ ]:
data_treat1 = data.loc[data['treatment'] == 0]

In [ ]:
no_resp_treat1_f = data_treat1.loc[data['gender'] == 0].was_response.value_counts()[0]
no_resp_treat1_m = data_treat1.loc[data['gender'] == 1].was_response.value_counts()[0]
resp_treat1_f = data_treat1.loc[data['gender'] == 0].was_response.value_counts()[1]
resp_treat1_m = data_treat1.loc[data['gender'] == 1].was_response.value_counts()[1]

In [ ]:
print(no_resp_treat1_f, no_resp_treat1_m, resp_treat1_f, resp_treat1_m)

117 119 24 32


In [ ]:
table4 = sm.stats.Table2x2(np.array([[resp_treat1_f, no_resp_treat1_f], [resp_treat1_m, no_resp_treat1_m]]))

In [ ]:
print(table4.summary(method='normal'))

               Estimate   SE   LCB    UCB  p-value
--------------------------------------------------
Odds ratio        0.763        0.424 1.373   0.366
Log odds ratio   -0.271 0.300 -0.858 0.317   0.366
Risk ratio        0.803        0.499 1.294   0.368
Log risk ratio   -0.219 0.243 -0.696 0.258   0.368
--------------------------------------------------


In [ ]:
print(f'Odds ratio of Female vs Male | treatment 1 is {np.exp(-0.271):.2f} with ({np.exp(-0.271 - z*0.300):.2f}, {np.exp(-0.271 + z*0.300):.2f})')

Odds ratio of Female vs Male | treatment 1 is 0.76 with (0.42, 1.37)


In [ ]:
data_treat2 = data.loc[data['treatment'] == 1]

In [ ]:
no_resp_treat2_f = data_treat2.loc[data['gender'] == 0].was_response.value_counts()[0]
no_resp_treat2_m = data_treat2.loc[data['gender'] == 1].was_response.value_counts()[0]
resp_treat2_f = data_treat2.loc[data['gender'] == 0].was_response.value_counts()[1]
resp_treat2_m = data_treat2.loc[data['gender'] == 1].was_response.value_counts()[1]

In [ ]:
print(no_resp_treat2_f, no_resp_treat2_m, resp_treat2_f, resp_treat2_m)

106 148 16 20


In [ ]:
table5 = sm.stats.Table2x2(np.array([[resp_treat2_f, no_resp_treat2_f], [resp_treat2_m, no_resp_treat2_m]]))

In [ ]:
print(table5.summary(method='normal'))

               Estimate   SE   LCB    UCB  p-value
--------------------------------------------------
Odds ratio        1.117        0.553 2.256   0.758
Log odds ratio    0.111 0.359 -0.592 0.814   0.758
Risk ratio        1.102        0.596 2.037   0.758
Log risk ratio    0.097 0.314 -0.518 0.711   0.758
--------------------------------------------------


In [ ]:
print(f'Odds ratio of Female vs Male | treatment 2 is {np.exp(0.111):.2f} with ({np.exp(0.111 - z*0.359):.2f}, {np.exp(0.111 + z*0.359):.2f})')

Odds ratio of Female vs Male | treatment 2 is 1.12 with (0.55, 2.26)


# Result from stast table2x2


*   Odds ratio of treatment 1 vs treatment 2 | Male is 1.99 with (1.08, 3.65)
*   Odds ratio of treatment 1 vs treatment 2 | Female is 1.36 with (0.69, 2.69)
*   Odds ratio of Female vs Male | treatment 1 is 0.76 with (0.42, 1.37)
*   Odds ratio of Female vs Male | treatment 2 is 1.12 with (0.55, 2.26)


# Logistic Regression model (sklearn)

Try to predict efficiency of therapy.

In [ ]:
X_3 = data.drop(columns=['was_response'])
y_3 = data.was_response

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_3, y_3, test_size=0.2, random_state=1, shuffle=True, stratify=y_3)

In [ ]:
model2 = LogisticRegression(penalty='l2', fit_intercept=False)
model2.fit(X_3, y_3)

LogisticRegression(fit_intercept=False)

In [ ]:
prediction2 = model2.predict(X_test)

In [ ]:
print(classification_report(y_test, prediction2))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92        99
           1       0.00      0.00      0.00        18

    accuracy                           0.85       117
   macro avg       0.42      0.50      0.46       117
weighted avg       0.72      0.85      0.78       117



In [ ]:
X_3.columns

Index(['gender', 'treatment', 'intercept', 'interaction'], dtype='object')

In [ ]:
model2.coef_

array([[ 0.17522894, -0.37704497, -1.50665951, -0.27735434]])

In [ ]:
X_2.columns

Index(['gender', 'treatment', 'intercept', 'interaction'], dtype='object')

In [ ]:
result_2.params.values

array([ 0.27073251, -0.30673027, -1.5841201 , -0.38136214])

# The result of working with data.

Data problem: data is binary, few features, unbalanced results.

It is better to use the Wilson interval for unbalanced data.

The resulting logistic regression model fails to predict a positive response to therapy. Therefore, the prediction of odds ratio using the coefficients from this model may not be entirely correct.

Based on the confidence intervals of the difference in proportions, it can be concluded that the effectiveness of treatment 1 and treatment 2 is statistically significantly different.

I think we should collect additional data to adjust the odds ratio calculations for different therapies for subgroups of patients.
